In [1]:
import env
import pandas as pd
import pyspark

from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, lit, when, asc, desc, expr, to_timestamp, datediff, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType

spark = pyspark.sql.SparkSession.builder.getOrCreate()

1. Read the case, department, and source data into their own spark dataframes.

In [2]:
source_df = spark.read.csv(env.data_path + "source.csv", sep=",", header=True, inferSchema=True)

In [3]:
case_df = spark.read.csv(env.data_path + "case.csv", sep=",", header=True, inferSchema=True)

In [4]:
dept_df = spark.read.csv(env.data_path + "dept.csv", sep=",", header=True, inferSchema=True)

2. Let's see how writing to the local disk works in spark:
* Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [5]:
source_df.write.json(env.data_path + "source_json", mode="overwrite")
source_df.write.csv(env.data_path + "source_csv", mode="overwrite")

In [6]:
case_df.write.json(env.data_path + "case_json", mode="overwrite")
case_df.write.csv(env.data_path + "case_csv", mode="overwrite")

In [7]:
dept_df.write.json(env.data_path + "dept_json", mode="overwrite")
dept_df.write.csv(env.data_path + "dept_csv", mode="overwrite")

* Inspect your folder structure. What do you notice?
> A folder with the specified name is created with several files inside of the specified file type.

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [8]:
source_df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



Looks good, but primary keys really shouldn't be nullable, but I could not figure out how to change it.

In [9]:
source_df.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [10]:
case_df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



* case_id should be converted to string values
* dates should be timestamps
* rename columns with uppercase characters
* YES/NO columns should be booleans
* council_district should be a string

In [11]:
case_df.show(5, vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [12]:
# Convert to string values
case_df = case_df.withColumn("case_id", case_df.case_id.cast("string"))
case_df = case_df.withColumn("council_district", case_df.council_district.cast("string"))

In [13]:
case_df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [14]:
# Rename columns with uppercase characters
case_df = case_df.withColumnRenamed('SLA_due_date', 'sla_due_date').withColumnRenamed('SLA_days', 'sla_days')

In [15]:
case_df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- sla_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- sla_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [16]:
# Convert YES/NO columns to booleans
case_df = case_df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn("case_late", expr('case_late == "YES"'))

In [17]:
case_df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- sla_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- sla_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [18]:
# Convert dates to timestamps
fmt = "M/d/yy H:mm"
case_df = (
    case_df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
    .withColumn("sla_due_date", to_timestamp("sla_due_date", fmt))
)

In [19]:
case_df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- sla_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- sla_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- case_due_date: timestamp (nullable = true)



In [20]:
dept_df.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



* rename dept_subject_to_SLA
* convert YES/NO column to boolean

In [21]:
dept_df.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [22]:
dept_df = dept_df.withColumnRenamed('dept_subject_to_SLA', 'dept_subject_to_sla')
dept_df = dept_df.withColumn("dept_subject_to_sla", expr('dept_subject_to_sla == "YES"'))

In [23]:
dept_df.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_sla: boolean (nullable = true)



1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [24]:
open_cases = case_df.where(case_df.case_closed == False)

In [25]:
open_cases.select("*").show(5, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014128388           
 case_opened_date     | 2018-01-02 09:39:00  
 case_closed_date     | 2018-01-02 09:39:00  
 sla_due_date         | 2018-01-09 09:39:00  
 case_late            | true                 
 num_days_late        | 211.5974884          
 case_closed          | false                
 dept_division        | 311 Call Center      
 service_request_type | Complaint            
 sla_days             | 7.0                  
 case_status          | Open                 
 source_id            | mt13131              
 request_address      | 7326  WESTGLADE P... 
 council_district     | 6                    
 case_due_date        | 2018-01-02 09:39:00  
-RECORD 1------------------------------------
 case_id              | 1014128790           
 case_opened_date     | 2018-01-02 10:49:00  
 case_closed_date     | 2018-01-02 10:49:00  
 sla_due_date         | 2018-05-10 10:49:00  
 case_late            | true      

In [28]:
open_cases = open_cases.withColumn('days_past_sla', datediff(current_timestamp(), 'sla_due_date'))\
.withColumn('days_open', datediff(current_timestamp(), 'case_opened_date'))

In [37]:
open_cases.sort(desc('days_past_sla')).show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | 2017-01-01 13:48:00  
 sla_due_date         | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 sla_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_due_date        | 2017-01-01 13:48:00  
 days_past_sla        | 1578                 
 days_open            | 1594                 
only showing top 1 row



In [38]:
open_cases.sort(desc('days_open')).show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | 2017-01-01 13:48:00  
 sla_due_date         | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 sla_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_due_date        | 2017-01-01 13:48:00  
 days_past_sla        | 1578                 
 days_open            | 1594                 
only showing top 1 row



2. How many Stray Animal cases are there?

In [51]:
case_df.where(expr("service_request_type = 'Stray Animal'")).count()

26760

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

4. Convert the council_district column to a string column.
5. Extract the year from the case_closed_date column.
6. Convert num_days_late from days to hours in new columns num_hours_late.
7. Join the case data with the source and department data.
8. Are there any cases that do not have a request source?
9. What are the top 10 service request types in terms of number of requests?
10. What are the top 10 service request types in terms of average days late?
11. Does number of days late depend on department?
12. How do number of days late depend on department and request type?